In [22]:
# 登陆邮箱并读取原始邮件
def get_mail(email_address, passsword):
    # 选择服务器
    server = imaplib.IMAP4_SSL('outlook.office365.com')
    server.login(email_address, passsword)
    inbox = server.select("INBOX")
    # 搜索匹配的邮件
    email_type, data = server.search(None, "ALL")
    # 邮件列表，使用空格分割得到邮件索引
    msglist = data[0].split()
    if len(msglist) != 0:
        # 最新邮件
        latest = msglist[len(msglist)-1]
        email_type, datas = server.fetch(latest, '(RFC822)')
        # 使用utf-8解码
        text = datas[0][1].decode('utf-8')
        # 转为email.message对象
        message = email.message_from_string(text)
        # 获取邮件数量
        server.select()
        email_count = len(server.search(None, 'ALL')[1][0].split())
        print('邮件一共有：', email_count)
        email_id_byte = server.search(None, 'ALL')[1][0].split()
        email_id = []
        for row in email_id_byte:
            email_id.append(row.decode('utf-8'))
        print(email_id)
        return message
    else:
        print("暂无邮件")

In [23]:
# 将原始邮件转化为可读邮件
def decode_str(s):
    value, charset = decode_header(s)[0]
    if charset:
        value = value.decode(charset)
    return value

In [24]:
import email
import imaplib
from email.utils import parseaddr
from email.header import decode_header #正确

In [25]:
def guess_charset(msg):
    charset = msg.get_charset()
    if charset is None:
        content_type = msg.get('Content-Type','').lower()
        pos = content_type.find('charset=')
        if pos >= 0:
            charset = content_type[pos + 8:].strip()
    return charset

In [56]:
text = []
def print_info(msg, indent=0):
    if indent == 0:
        for header in ['From', 'To', 'Subject']:
            value = msg.get(header, '')
            if value:
                if header == 'Subject':
                    value = decode_str(value)
                else:
                    hdr, addr = parseaddr(value)
                    name = decode_str(hdr)
                    value = u'%s <%s>' % (name, addr)
            print('%s%s: %s' % ('  ' * indent, header, value))

    if msg.is_multipart():
        parts = msg.get_payload()
        for n, part in enumerate(parts):
            #print('%spart %s' % ('  ' * indent, n))
            #print('%s--------------------' % ('  ' * indent))
            print_info(part, indent + 1)
    else:
        content_type = msg.get_content_type()
        if content_type == 'text/plain' or content_type == 'text/html':
            content = msg.get_payload(decode=True)
            charset = guess_charset(msg)
            if charset:
                content = content.decode(charset)
            text1 = '%sText: %s' % ('  ' * indent, content + '...')
            text.append(text1)
            print(text)
        else:
            print('%sAttachment: %s' % ('  ' * indent, content_type))


In [57]:
if __name__ == "__main__":
    email_addr = "charmingzj@outlook.com"
    password = "xvbxscoccmlgjnxp"
    messageObject = get_mail(email_addr, password)
    if messageObject:
        msgDate = messageObject["date"]
        print("发送时间:%s" % msgDate)
        res = print_info(messageObject)

邮件一共有： 5
['1', '2', '3', '4', '5']
发送时间:Thu, 2 Feb 2023 20:19:02 +0000
From:  <gtd@start.umd.edu>
To:  <charmingzj@outlook.com>
Subject: Global Terrorism Database Download
['  Text: Thank you for your interest in the Global Terrorism Database. You submitted the following information with your request to download the full data file:\r\nFirst Name: Zeng\r\nLast Name:  Jin\r\nEmail:  charmingzj@outlook.com\r\nInstitution / Affiliation:  MUST\r\nCountry:  China\r\nType of Use:  academic\r\n\r\nUse Case:\r\nTo do data analysis for assignment.\r\n\r\nEULA: I have read and agree to the terms of the EULA.\r\nConsent: I have read the privacy notice and I grant permission for the University of Maryland to use my information for legitimate purposes.\r\n\r\nTo access the data file and documentation, please click the following link or paste it into your browser:\r\nhttps://www.start.umd.edu/gtd/contact/download?t=d543d7d2a33611ed847c0e5194896103\r\nThis link will expire in 24 hours, or after the da

In [ ]:
#把ipynb文件转化为.py文件方便调用
#在command line输入：jupyter nbconvert --to python Tools/Get_email.ipynb